In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!unrar x /content/gdrive/MyDrive/denemekaps3.rar

In [ ]:
from google.colab import files
files.upload()

In [ ]:
def load_data():
    yol = '/content/denemekaps/'
    path = yol
    data = []
    label = []
    Files = ['herniyok', 'resim2']
    label_val = 0

    for files in Files:
        cpath = os.path.join(path, files)
        cpath = os.path.join(cpath)

        for img in sorted(os.listdir(cpath)):

          image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
          image_array=cv2.resize(image_array,(320,320))
          image_array=np.uint8(image_array)
          data.append(image_array)
          label.append(label_val)
        label_val = 1

    data=np.asarray(data)
    label =np.asarray(label,dtype=np.uint8)

    data = data.reshape(-1, 320, 320,3 ).astype('float32') / 255.
    label = to_categorical(label.astype('float32'))

    return data, label

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

data, label = load_data()

# Parametreler
weight_path_template = "{}_densenet_fold_{}.weights.h5"
epochs = 50
batch_size = 4
num_classes = len(np.unique(label))

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Sonuçları saklamak için
fold_train_accuracies = []
fold_val_accuracies = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f"\n--- Fold {fold + 1} ---")

    X_train, X_val = data[train_idx], data[val_idx]
    y_train, y_val = label[train_idx], label[val_idx]

    input_shape = (320, 320, 3)
    input_layer = Input(shape=input_shape)
    base_model = DenseNet169(include_top=False, input_shape=input_shape, weights='imagenet')
    x = base_model(input_layer)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    output_layer = Dense(2, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)


    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    weight_path = weight_path_template.format("BEST", fold + 1)
    checkpoint = ModelCheckpoint(weight_path, monitor='loss', verbose=1,
                                 save_best_only=True, mode='min', save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4,
                                  verbose=1, mode='min', min_lr=0.0001)
    callbacks_list = [checkpoint]

    history = model.fit(
        X_train.astype(np.float32), y_train.astype(np.float32),
        batch_size=batch_size,
        shuffle=True,
        epochs=epochs,
        callbacks=callbacks_list
    )

    train_accuracy = max(history.history['accuracy'])
    fold_train_accuracies.append(train_accuracy)

    print(f"Fold {fold + 1} Train Accuracy: {train_accuracy:.4f}")

    y_val_pred = np.argmax(model.predict(X_val), axis=1)
    y_val_int = np.argmax(y_val, axis=1)


    accuracy = accuracy_score(y_val_int, y_val_pred)
    precision = precision_score(y_val_int, y_val_pred, average='weighted')
    recall = recall_score(y_val_int, y_val_pred, average='weighted')
    f1 = f1_score(y_val_int, y_val_pred, average='weighted')

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f"Fold {fold + 1} Metrics:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")

print("\n--- Cross-Validation Results ---")
print(f"Average Train Accuracy: {np.mean(fold_train_accuracies):.4f}")
print(f"Average Validation Accuracy: {np.mean(fold_val_accuracies):.4f}")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")
